In [1]:
!nvidia-smi

Mon Feb 16 18:34:22 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   59C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# --- CLEAN START ---
# Remove everything that can conflict
!pip uninstall -y unsloth unsloth-zoo transformers accelerate peft datasets tokenizers huggingface-hub bitsandbytes sentencepiece

# --- PYTORCH (CUDA 11.8) ---
!pip install torch --index-url https://download.pytorch.org/whl/cu118

# --- INSTALL MODERN UNSLOTH (the correct one for Qwen2.5) ---
!pip install "unsloth==2026.2.1"

# --- INSTALL THE HF STACK COMPATIBLE WITH UNSLOTH 2026.x ---
!pip install "transformers>=4.57.0" "accelerate>=0.34.1" "peft>=0.18.0" datasets sentencepiece bitsandbytes

# --- OPTIONAL EXTRAS ---
!pip install wandb plotly

Found existing installation: transformers 5.0.0
Uninstalling transformers-5.0.0:
  Successfully uninstalled transformers-5.0.0
Found existing installation: accelerate 1.12.0
Uninstalling accelerate-1.12.0:
  Successfully uninstalled accelerate-1.12.0
Found existing installation: peft 0.18.1
Uninstalling peft-0.18.1:
  Successfully uninstalled peft-0.18.1
Found existing installation: datasets 4.0.0
Uninstalling datasets-4.0.0:
  Successfully uninstalled datasets-4.0.0
Found existing installation: tokenizers 0.22.2
Uninstalling tokenizers-0.22.2:
  Successfully uninstalled tokenizers-0.22.2
Found existing installation: huggingface_hub 1.4.0
Uninstalling huggingface_hub-1.4.0:
  Successfully uninstalled huggingface_hub-1.4.0
Found existing installation: sentencepiece 0.2.1
Uninstalling sentencepiece-0.2.1:
  Successfully uninstalled sentencepiece-0.2.1
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 4.3 MB/s eta 0:00:00

In [3]:
%cd /content
!rm -rf detective_finetune_lab

!git clone https://github.com/carloderossi/detective_finetune_lab.git
%cd detective_finetune_lab


!ls -R


/content
Cloning into 'detective_finetune_lab'...
remote: Enumerating objects: 462, done.
remote: Counting objects: 100% (462/462), done.
remote: Compressing objects: 100% (364/364), done.
remote: Total 462 (delta 93), reused 423 (delta 54), pack-reused 0 (from 0)
Receiving objects: 100% (462/462), 8.45 MiB | 14.57 MiB/s, done.
Resolving deltas: 100% (93/93), done.
/content/detective_finetune_lab
.:
CDR_OLLAMA_Serve.cmd  pyproject.toml  reports  SomeIMPROVEMENTS.md
data		      README.md       scripts

./data:
cleaned  evaluation  raw  rl  train

./data/cleaned:
agatha_christie  arthur_conan_doyle

./data/cleaned/agatha_christie:
poirot_investigates_by_agatha_christie
the_big_four_by_agatha_christie
the_murder_of_roger_ackroyd_by_agatha_christie
the_mysterious_affair_at_styles_by_agatha_christie
the_mystery_of_the_blue_train_by_agatha_christie

./data/cleaned/agatha_christie/poirot_investigates_by_agatha_christie:
chapter_01.txt	chapter_04.txt	chapter_07.txt	chapter_10.txt
chapter_02.tx

In [ ]:
!python scripts/train_scout_unsloth.py

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
2026-02-16 18:36:52.814618: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771267013.006387    1263 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771267013.069093    1263 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771267013.515901    1263 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771267013.515939    1263 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:17

🦥 Unsloth Zoo will now patch everything to make training faster!
Loading model Qwen/Qwen2.5-7B-Instruct with Unsloth...
==((====))==  Unsloth 2026.2.1: Fast Qwen2 patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
model.safetensors.index.json: 112kB [00:00, 194MB/s]
model-00001-of-00002.safetensors: 100% 4.99G/4.99G [00:37<00:00, 134MB/s]     
model-00002-of-00002.safetensors: 100% 2.16G/2.16G [00:24<00:00, 86.7MB/s]   
Loading checkpoint shards: 100% 2/2 [00:25<00:00, 12.88s/it]
generation_config.json: 100% 271/271 [00:00<00:00, 1.67MB/s]
tokenizer_config.json: 7.36kB [00:00, 30.0MB/s]
vocab.json: 2.78MB [00:00, 72.2MB/s]
merges.txt: 